# Maximum Likelihood Estimation
This notebook assumes you know a little bit about maximum likelihood estimation. It demonstrates how to do MLE in `Julia`.

## Goal
1. Simulate data and run the maximum likelihood estimation procedure.
2. Test the results using ordinary least squares.

In [1]:
VERSION

v"1.0.5"

In [2]:
# Loading the required libraries
using Distributions # to generate random numbers from distributions
using Random # to set the seed for reproducibility
using Optim # for optimization routines 
using DataFrames # to provide DataFrame input to the lm function
using GLM # for ordinary least squares

In [3]:
# Set seed
rng = MersenneTwister(1234);

In [12]:
# Intercept
x1 = ones(1000)
# X variable
x2 = rand(Uniform(0, 10), 1000)
# Binding them together
x = hcat(x1, x2)
# True parameter values
true_theta = [2, 4, 6] # error variance = 2, intercept = 4, slope = 6
# Generating the dependent variables
y = x * true_theta[2:3] + sqrt(true_theta[1]) * randn(1000)
# View the first 6 
y[1:6]

1000-element Array{Float64,1}:
 31.086354268466064 
 13.043152756069253 
 37.29743911326745  
 34.98854885421249  
  6.46759187522878  
 59.65096072357595  
 46.70962694632156  
 11.239034236922034 
  3.0022492658235107
 25.15012309016758  
 31.358447608709515 
 33.271288503213555 
 59.84850876217838  
  ⋮                 
  9.357428468345715 
 15.061593500283127 
  6.203767989866787 
 12.842100075209215 
 52.2075437339323   
 12.178972455462027 
  2.3222961772438997
 24.250379220534484 
 38.48735686535129  
 44.10078376117888  
  9.565476505742422 
 61.88612353982749  

In [47]:
# Writing down the likelihood function
function likelihood(theta, y, x)
   β = theta[2:length(theta)]
   σ = theta[1]
    if σ^2 <= 0 
        return missing
    else
       n = size(x)[1]
       e = y - x * β
        logL = ((-n/2)*log(2*π)) - ((n/2)*log(σ^2))  - e'*e/(2*σ^2)
        return -logL
    end
end

likelihood (generic function with 1 method)

## Gradient free approach
### Using the Nelder Mead algorithm

In [63]:
## Optimize over theta, give the starting values, and the name of the algorithm
nmopt = optimize(z -> likelihood(z, y, x), [1., 1., 1.], NelderMead())
### ----------------- Methods to extract useful information from the results
## Extract the minimizers using
@show nmopt.minimizer ## alternatively Optim.minimizer(nmopt)
## Extract the minimum value using 
@show nmopt.minimum ## alternatively, Optim.minimum(nmopt)
## Extract the summary using
@show summary(nmopt) ## alternatively, Optim.summary(nmopt)
## Extract the convergence status using
@show Optim.converged(nmopt)
## Similar methods exist for other algorithms.

nmopt.minimizer = [1.37542, 3.8673, 6.02626]
nmopt.minimum = 1737.7011389796992
summary(nmopt) = "Nelder-Mead"
Optim.converged(nmopt) = true


true

## Test against OLS
Here, we run a simple OLS regression to check if the results from our MLE optimization give us the same answer as the ordinary leasy squares.

In [65]:
# Get a DataFrame
dat = DataFrame(Y=y, X2=x2)
# Run OLS
ols = lm(@formula(Y ~ X2), dat)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Y ~ 1 + X2

Coefficients:
───────────────────────────────────────────────────────────────────────────
             Estimate  Std. Error   t value  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   3.86731   0.0850756   45.4573    <1e-99    3.70036    4.03425
X2            6.02626   0.0151822  396.93      <1e-99    5.99647    6.05605
───────────────────────────────────────────────────────────────────────────

In [79]:
?rounding

search: rounding RoundingMode setrounding groupindices RoundNearestTiesUp



```
rounding(T)
```

Get the current floating point rounding mode for type `T`, controlling the rounding of basic arithmetic functions ([`+`](@ref), [`-`](@ref), [`*`](@ref), [`/`](@ref) and [`sqrt`](@ref)) and type conversion.

See [`RoundingMode`](@ref) for available modes.


In [70]:
@show nmopt.minimizer[2:3]
@show coef(ols)

nmopt.minimizer[2:3] = [3.8673, 6.02626]
coef(ols) = [3.86731, 6.02626]


2-element Array{Float64,1}:
 3.8673072523507  
 6.026261119981097

In [81]:
round(coef(ols))

MethodError: MethodError: no method matching round(::Array{Float64,1})
Closest candidates are:
  round(!Matched::Type{BigInt}, !Matched::BigFloat) at mpfr.jl:258
  round(!Matched::Float64, !Matched::RoundingMode{:Nearest}) at float.jl:368
  round(!Matched::Float64, !Matched::RoundingMode{:Up}) at float.jl:366
  ...

In [80]:
round(coef(ols), digits=4) == round(nmopt.minimizer[2:3], digits=4)

MethodError: MethodError: no method matching round(::Array{Float64,1}; digits=4)
Closest candidates are:
  round(!Matched::Type{BigInt}, !Matched::BigFloat) at mpfr.jl:258 got unsupported keyword argument "digits"
  round(!Matched::Float64, !Matched::RoundingMode{:Nearest}) at float.jl:368 got unsupported keyword argument "digits"
  round(!Matched::Float64, !Matched::RoundingMode{:Up}) at float.jl:366 got unsupported keyword argument "digits"
  ...

In [31]:
using GLM

In [34]:
using DataFrames

In [46]:
convert(Vector{Float64}, x2)
convert(Vector{Float64}, y)

1000-element Array{Float64,1}:
 31.086354268466064 
 13.043152756069253 
 37.29743911326745  
 34.98854885421249  
  6.46759187522878  
 59.65096072357595  
 46.70962694632156  
 11.239034236922034 
  3.0022492658235107
 25.15012309016758  
 31.358447608709515 
 33.271288503213555 
 59.84850876217838  
  ⋮                 
  9.357428468345715 
 15.061593500283127 
  6.203767989866787 
 12.842100075209215 
 52.2075437339323   
 12.178972455462027 
  2.3222961772438997
 24.250379220534484 
 38.48735686535129  
 44.10078376117888  
  9.565476505742422 
 61.88612353982749  